<div class="alert alert-success">
<b>Комментарий ревьюера v 2.0 ✔️:</b> Отлично, Ольга! В твоем проекте отсутствуют значительные недостатки и он конечно же принят. Спасибо за работу!)

# Исследование надежности заемщиков.

Цель исследования - проверить 2 гипотезы:

1. Влияет ли семейное положение на факт погашения кредита в срок?

2. Влияет ли количество детей клиента на факт погашения кредита в срок.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')

df.head(10) #выведу первые 10 строк датафрейма

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В столбце days_employed присутствую отрицательные значения, что странно для трудового стажа, и экстремально большие значения, которые не похожи на дни. Предположу, что это часы, и перевуду их в дни. В столбце education присутствуют неявные дубликаты.

In [2]:
df.info() #посмотрим общую информацию о датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Всего в датафрейме 12 столбцов и 21525 строк. Метод info() выявил пропуски в столбцах days_employed и total_income. Причем одинаковое количество. Названия столбцов написаны корректно.

### Шаг 2.1 Заполнение пропусков

In [3]:
df.isna().sum() #найду пропуски и посчитаю их количество

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах days_employed и total_income по 2174 пропущенных значений. Значения имеют тип float64 и относятся к количественным переменным.

In [4]:
'Доля пропущенных значений в столбце days_employed: {:.0%}'.format(df['days_employed'].isna().sum() / len(df)) #проверю, какую долю составляют пропущенные значения в каждом из столбцов с пропусками

'Доля пропущенных значений в столбце days_employed: 10%'

In [5]:
'Доля пропущенных значений в столбце total_income: {:.0%}'.format(df['total_income'].isna().sum() / len(df))

'Доля пропущенных значений в столбце total_income: 10%'

Доля пропущенных значений в каждом из столбцов с пропусками составляет 10%. Возможно, это люди, у которых еще нет трудового стажа и, соответсвенно, ежемесячного дохода. Проверим.

In [6]:
df.loc[df['days_employed'].isna()].head(10) #выведу первые 10 строк с пропусками в столбцах days_employed и total_income

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Возраст варьируется от 21 до 65 лет. Вряд ли все эти люди не имеют трудового стажа. Значит, заполню пропуски. Возьму медиану, т.к. значения в столбцах сильно разнятся.

In [7]:
total_income_avg = df['total_income'].median() #создаю переменную для медианы total_income
df.loc[:,'total_income'] = df.loc[:,'total_income'].fillna(total_income_avg) #заполняю пропуски медианой
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В столбце total_income пропусков нет.

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [8]:
df['days_employed'] = df['days_employed'].abs() #уберу знак '-' у отрицательных значений
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Возможно данные выгрузились некорректно.

In [9]:
days_employed_avg = df['days_employed'].median() #создаю переменную для медианы days_employed
df.loc[:,'days_employed'] = df.loc[:,'days_employed'].fillna(days_employed_avg) #заполняю пропуски медианой
df['days_employed'] = df['days_employed'].astype(int) #заменю вещественный тип данных в столбце days_employed на целочисленный
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


В столбце days_employed пропусков нет, тип переменной изменился на int64.

Так же в столбце days_employed есть эктремально большие значения. Предположу, что это часы и переведу их в дни.

In [10]:
df['days_employed'] = df['days_employed'].apply(lambda x: x/24 if x >300000 else x) #переведу часы в дни
df.loc[df['days_employed']>=300000]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Экстремально большие значения убрали.

### Шаг 2.3. Изменение типов данных.

In [11]:
df['total_income'] = df['total_income'].astype(int) #заменю вещественный тип данных в столбце total_income на целочисленный
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Тип переменной в столбце изменился total_income на int64.

### Шаг 2.4. Удаление дубликатов.

In [12]:
df.duplicated().sum() #найду явные дубликаты и посчитаю такие строки

54

In [13]:
df.drop_duplicates().reset_index(drop=True).info() #удалю строки-дубликаты

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21471 non-null  int64  
 1   days_employed     21471 non-null  float64
 2   dob_years         21471 non-null  int64  
 3   education         21471 non-null  object 
 4   education_id      21471 non-null  int64  
 5   family_status     21471 non-null  object 
 6   family_status_id  21471 non-null  int64  
 7   gender            21471 non-null  object 
 8   income_type       21471 non-null  object 
 9   debt              21471 non-null  int64  
 10  total_income      21471 non-null  int64  
 11  purpose           21471 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Методом drop_duplicates() мы удалили дублирующие строки. Количество строк уменьшилось на 54.

In [14]:
df['children'].value_counts() #проверю столбец children

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Для поиска дубликатов я использовала метод value_counts(), который анализирует столбец, выбирает каждое уникальное значение и подсчитывает частоту его встречаемости в списке.

В столбце children выделяются два странных значения 20 и -1. Предположу, что это человеческий фактор и заменю 20 на 2, -1 на 1.

In [15]:
df['children'] = df['children'].replace(20, 2) #заменю аномальные значения в столбце children
df['children'] = df['children'].replace(-1, 1)
df['children'].value_counts() #проверю предыдущее действие

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [16]:
df['education'].value_counts() #проверю столбец education

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Я вижу 3 варианта записи в этом столбце. Возможно таблицу заполняли 3 разных человека.

In [17]:
df['education'] = df['education'].str.lower() #приведу все значения в столбце education к единому регистру
df['education'].value_counts() #проверю предыдущее действие

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [18]:
df['family_status'].value_counts() #проверю столбец family_status

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [19]:
df['family_status'] = df['family_status'].str.lower() #приведу все значения в столбце family_status к единому регистру
df['family_status'].value_counts() #проверю предыдущее действие

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [20]:
df['income_type'].value_counts() #проверю столбец income_type

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Тут все хорошо.

In [21]:
df['purpose'].value_counts() #проверю столбец purpose

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

В столбце purpose много одинаковых операций записаны по-разному. Категоризирую их.
Скорее всего информацию писали разные люди, и каждый записывал по-своему.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [22]:
education_log = (df[['education','education_id']]) #создаю новый датафрейм для education
education_log.drop_duplicates().reset_index(drop=True)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [23]:
family_status_log = df[['family_status','family_status_id']] #создаю новый датафрейм для family_status
family_status_log = family_status_log.drop_duplicates().reset_index(drop=True)
family_status_log

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [24]:
df = df.drop(['education', 'family_status'], axis=1) #удаляю столбцы education и family_status
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.00,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.00,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.00,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.00,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177.75,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.00,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879.00,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152.00,50,1,0,M,сотрудник,0,135823,образование
8,2,6929.00,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.00,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.6. Категоризация дохода.

In [25]:
def total_income_category(total_income): #создаю функцию, которая категоризирует данные в столбце total_income
    
    if total_income <= 30000:
        return 'D'
    
    if 30001 <= total_income <= 50000:
        return 'E'
    
    if 50001 <= total_income <= 200000:
        return 'C'
    
    if 200001 <= total_income <= 1000000:
        return 'B'
    
    return 'A'

df['total_income_category'] = df['total_income'].apply(total_income_category) #создаю столбец total_income_category с категориями

df.head(10) #выведу первые 10 строк датафрейма

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.00,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.00,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.00,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.00,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177.75,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926.00,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879.00,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152.00,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929.00,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188.00,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


In [26]:
df['total_income_category'].value_counts() #выведу статистику по ежемесячному доходу

C    16087
B     5041
E      350
A       25
D       22
Name: total_income_category, dtype: int64

### Шаг 2.7. Категоризация целей кредита.

In [28]:
def purpose_category(item): #создаю функцию, которая категоризирует данные в столбце purpose
    
    if 'свадьб' in item:
        return 'проведение свадьбы'
    if 'образован' in item:
        return 'получение образования'
    if 'автомобил' in item:
        return 'операции с автомобилем'
    if 'жиль' or 'недвижимост' in item:
        return 'операции с недвижимостью'
    return 'нет категории'
 
df['purpose_category'] = df['purpose'].apply(purpose_category) #создаю столбец purpose с категориями
df.head(10) #выведу первые 10 строк датафрейма

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.00,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.00,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.00,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.00,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177.75,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.00,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.00,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.00,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.00,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.00,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


In [ ]:
df['purpose_category'].value_counts() #выведу статистику по ежемесячному доходу

### Ответы на вопросы.

##### Вопрос 1: 

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
df_pivot_children = df.pivot_table(index = 'children', columns = 'debt', values = 'dob_years', aggfunc = 'count') #создам сводную таблицу
df_pivot_children['%'] = df_pivot_children[1]/(df_pivot_children[0]+df_pivot_children[1])*100 #посчитаю долю непогашенных кредитов
df_pivot_children.sort_values(by='%', ascending=False) #выведу таблицу в порядке убывания по столбцу %

##### Вывод 1:

Семьи с 5 детьми не имеют задолженностей. Дальше мы видим, как с увеличением количества детей, растет и процент задолженностей. При это семьи с 3 детьми выплачивают кредит в срок чаще, чем семьи с одним ребенком. Данные неоднозначные.

##### Вопрос 2: 

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
df_pivot_family_status = df.pivot_table(index = 'family_status_id', columns = 'debt', values = 'dob_years', aggfunc = 'count') #создам сводную таблицу
df_pivot_family_status['%'] = df_pivot_family_status[1]/(df_pivot_family_status[0]+df_pivot_family_status[1])*100 #посчитаю долю непогашенных кредитов
df_pivot_family_status.sort_values(by='%', ascending=False) #выведу таблицу в порядке убывания по столбцу %

In [ ]:
df_pivot_family_status_new = df_pivot_family_status.merge(family_status_log, on='family_status_id', how='left') #объединю полученную таблицу со словарем family_status
df_pivot_family_status_new.sort_values(by='%', ascending=False) #выведу таблицу в порядке убывания по столбцу %

##### Вывод 2:

Неженатые/незамужние имеют самый большой процент задолженностей. Те, кто развелся или овдовел - самый маленький. Здесь тоже сложно говорить о какой-либо зависимости.

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
df_pivot_total_income = df.pivot_table(index = 'total_income_category', columns = 'debt', values = 'dob_years', aggfunc = 'count') #создам сводную таблицу
df_pivot_total_income['%'] = df_pivot_total_income[1]/(df_pivot_total_income[0]+df_pivot_total_income[1])*100 #посчитаю долю непогашенных кредитов
df_pivot_total_income.sort_values(by='%', ascending=False) #выведу таблицу в порядке убывания по столбцу %

##### Вывод 3:

Получается, что люди с наименьшим ежемесячным доходом выплачивают кредит в срок чаще, чем те, у кого ежемесячный доход высокий, что довольно странно. Самый большой процент задолженностей по кредитам у средней категории.

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

In [ ]:
df_pivot_purpose = df.pivot_table(index = 'purpose_category', columns = 'debt', values = 'dob_years', aggfunc = 'count') #создам сводную таблицу
df_pivot_purpose['%'] = df_pivot_purpose[1]/(df_pivot_purpose[0]+df_pivot_purpose[1])*100 #посчитаю долю непогашенных кредитов
df_pivot_purpose.sort_values(by='%', ascending=False) #выведу таблицу в порядке убывания по столбцу %

##### Вывод 4:

Мы видим, что на операции с автомобилем и получение образования процент задолженностей по кредиту выше, чем на операции с недвижимостью и проведение свадеб.

## Общий вывод:

Была поставлена задача разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

На основе полученных данных, я сформировала новые датафреймы по конкретным категориям и посчитала долю непогашенных в срок кредитов для каждой категории.

Полученные данные не дали мне однозначного ответа на поставленный вопрос. Думаю, данные неоднозначны, чтобы делать по ним вывод.